In [ ]:
!pip install transformers peft datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
## to install llama2-70b
from huggingface_hub import notebook_login
notebook_login()
token = "your hf token here"

In [ ]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate
!pip install bitsandbytes

In [ ]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import csv
# import accelerate
# import bitsandbytes as bnb

# # Load the saved model and tokenizer
model_path = "/content/drive/MyDrive/01-Research/Geo-Isa/Results/Mistral/LoRA/GeoIsa_LoRA/final_model"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# # Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Function to extract subject-predicate-object from a tweet
def extract_spo_from_tweet(tweet):
    instruction_predicate = "Identify the verb or verb phrase that describes the geographical relationship between the two entities in the sentence."
    instruction_entity1= "Identify the entity that is the starting location or reference point for the geographical relationship described by the predicate. This entity is usually the subject of the sentence. Include the entire noun or noun phrase that represents the entity, and do not include any additional information about the entity."
    instruction_entity2= "Identify the entity that is the destination or endpoint for the geographical relationship described by the predicate. This entity is usually the object of the sentence. Include the entire noun or noun phrase that represents the entity, and do not include any additional information about the entity."

    input_predicate = f"User: {instruction_predicate} {tweet}\nAssistant:"
    input_entity1 = f"User: {instruction_entity1} {tweet}\nAssistant:"
    input_entity2 = f"User: {instruction_entity2} {tweet}\nAssistant:"

    input_ids_predicate = tokenizer(input_predicate, return_tensors='pt').input_ids
    input_ids_entity1 = tokenizer(input_entity1, return_tensors='pt').input_ids
    input_ids_entity2 = tokenizer(input_entity2, return_tensors='pt').input_ids

    max_new_tokens = 50  # You can adjust this value based on your requirements

    output_predicate = model.generate(input_ids_predicate, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    output_entity1 = model.generate(input_ids_entity1, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    output_entity2 = model.generate(input_ids_entity2, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)

    predicate = tokenizer.decode(output_predicate[0], skip_special_tokens=True).split("Assistant:")[-1].strip()
    entity1 = tokenizer.decode(output_entity1[0], skip_special_tokens=True).split("Assistant:")[-1].strip()
    entity2 = tokenizer.decode(output_entity2[0], skip_special_tokens=True).split("Assistant:")[-1].strip()

    return {
        "tweet": tweet,
        "predicate": predicate,
        "subject": entity1,
        "object": entity2
    }

# Function to process all tweets in a file and save to CSV
def process_tweets(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(["tweet", "predicate", "", "subject", "object"])
        lines = infile.readlines()
        for line in tqdm(lines, desc="Processing tweets"):
            tweet = line.strip()
            if tweet:
                spo_triple = extract_spo_from_tweet(tweet)
                csv_writer.writerow([spo_triple["tweet"], spo_triple["predicate"], "", spo_triple["subject"], spo_triple["object"]])

# Example usage
# tweet = "Highland Park is in New Jersey"
# spo_triple = extract_spo_from_tweet(tweet)
# print(spo_triple)

# # Example file processing
input_file = "/content/drive/MyDrive/01-Research/Geo-Isa/Coding/evaluation-1600-annotated-tweets.txt"
output_file = "/content/drive/MyDrive/01-Research/Geo-Isa/Coding/Mistral_evaluation_1600_annotated_tweets.csv"
process_tweets(input_file, output_file)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Processing tweets: 100%|██████████| 1672/1672 [6:23:45<00:00, 13.77s/it]
